In [1]:
!pip install pandas

You should consider upgrading via the 'C:\Users\Cameron\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [2]:
import pandas as pd

Before we start to create the mechanisms behind caluculating the elo scores, we first have to find columns which will be of use to us! Let's load in the drivers and results files.

In [3]:
drivers_df = pd.read_csv("data/drivers.csv")
drivers_df.head()

,driverId,driverRef,number,code,forename,surname,dob,nationality,url
0,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton
1,2,heidfeld,\N,HEI,Nick,Heidfeld,1977-05-10,German,http://en.wikipedia.org/wiki/Nick_Heidfeld
2,3,rosberg,6,ROS,Nico,Rosberg,1985-06-27,German,http://en.wikipedia.org/wiki/Nico_Rosberg
3,4,alonso,14,ALO,Fernando,Alonso,1981-07-29,Spanish,http://en.wikipedia.org/wiki/Fernando_Alonso
4,5,kovalainen,\N,KOV,Heikki,Kovalainen,1981-10-19,Finnish,http://en.wikipedia.org/wiki/Heikki_Kovalainen


In [4]:
drivers_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 857 entries, 0 to 856
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   driverId     857 non-null    int64 
 1   driverRef    857 non-null    object
 2   number       857 non-null    object
 3   code         857 non-null    object
 4   forename     857 non-null    object
 5   surname      857 non-null    object
 6   dob          857 non-null    object
 7   nationality  857 non-null    object
 8   url          857 non-null    object
dtypes: int64(1), object(8)
memory usage: 60.4+ KB


Columns of interest: driverId, driverRef, surname

In [5]:
drivers_df = drivers_df[['driverId','driverRef','surname']]
drivers_df.head()

,driverId,driverRef,surname
0,1,hamilton,Hamilton
1,2,heidfeld,Heidfeld
2,3,rosberg,Rosberg
3,4,alonso,Alonso
4,5,kovalainen,Kovalainen


Noice. We've got the columns we need from the drivers dataset.

In [6]:
constructors_df = pd.read_csv('data/constructors.csv')
constructors_df.head()

,constructorId,constructorRef,name,nationality,url
0,1,mclaren,McLaren,British,http://en.wikipedia.org/wiki/McLaren
1,2,bmw_sauber,BMW Sauber,German,http://en.wikipedia.org/wiki/BMW_Sauber
2,3,williams,Williams,British,http://en.wikipedia.org/wiki/Williams_Grand_Pr...
3,4,renault,Renault,French,http://en.wikipedia.org/wiki/Renault_in_Formul...
4,5,toro_rosso,Toro Rosso,Italian,http://en.wikipedia.org/wiki/Scuderia_Toro_Rosso


In [7]:
constructors_df = constructors_df[['constructorId','constructorRef']]
constructors_df.head()

,constructorId,constructorRef
0,1,mclaren
1,2,bmw_sauber
2,3,williams
3,4,renault
4,5,toro_rosso


In [8]:
results_df = pd.read_csv("data/results.csv")
results_df.head()

,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,laps,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId
0,1,18,1,1,22,1,1,1,1,10.0,58,1:34:50.616,5690616,39,2,1:27.452,218.300,1
1,2,18,2,2,3,5,2,2,2,8.0,58,+5.478,5696094,41,3,1:27.739,217.586,1
2,3,18,3,3,7,7,3,3,3,6.0,58,+8.163,5698779,41,5,1:28.090,216.719,1
3,4,18,4,4,5,11,4,4,4,5.0,58,+17.181,5707797,58,7,1:28.603,215.464,1
4,5,18,5,1,23,3,5,5,5,4.0,58,+18.014,5708630,43,1,1:27.418,218.385,1


In [9]:
results_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25840 entries, 0 to 25839
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   resultId         25840 non-null  int64  
 1   raceId           25840 non-null  int64  
 2   driverId         25840 non-null  int64  
 3   constructorId    25840 non-null  int64  
 4   number           25840 non-null  object 
 5   grid             25840 non-null  int64  
 6   position         25840 non-null  object 
 7   positionText     25840 non-null  object 
 8   positionOrder    25840 non-null  int64  
 9   points           25840 non-null  float64
 10  laps             25840 non-null  int64  
 11  time             25840 non-null  object 
 12  milliseconds     25840 non-null  object 
 13  fastestLap       25840 non-null  object 
 14  rank             25840 non-null  object 
 15  fastestLapTime   25840 non-null  object 
 16  fastestLapSpeed  25840 non-null  object 
 17  statusId    

In [10]:
results_df = results_df[['resultId','driverId','constructorId','grid','position','positionText','positionOrder']]
results_df.head()

,resultId,driverId,constructorId,grid,position,positionText,positionOrder
0,1,1,1,1,1,1,1
1,2,2,2,5,2,2,2
2,3,3,3,7,3,3,3
3,4,4,4,11,4,4,4
4,5,5,1,3,5,5,5


Awesome! Now we have all the columns that we need. All that is left to do is merge the two dataframes together into one dataset.

In [11]:
results_df = pd.merge(results_df, drivers_df, how='left', on='driverId')
results_df.head()

,resultId,driverId,constructorId,grid,position,positionText,positionOrder,driverRef,surname
0,1,1,1,1,1,1,1,hamilton,Hamilton
1,2,2,2,5,2,2,2,heidfeld,Heidfeld
2,3,3,3,7,3,3,3,rosberg,Rosberg
3,4,4,4,11,4,4,4,alonso,Alonso
4,5,5,1,3,5,5,5,kovalainen,Kovalainen


In [12]:
results_df = pd.merge(results_df, constructors_df, how='left', on='constructorId')
results_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25840 entries, 0 to 25839
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   resultId        25840 non-null  int64 
 1   driverId        25840 non-null  int64 
 2   constructorId   25840 non-null  int64 
 3   grid            25840 non-null  int64 
 4   position        25840 non-null  object
 5   positionText    25840 non-null  object
 6   positionOrder   25840 non-null  int64 
 7   driverRef       25840 non-null  object
 8   surname         25840 non-null  object
 9   constructorRef  25840 non-null  object
dtypes: int64(5), object(5)
memory usage: 2.2+ MB


In [13]:
results_df.head()

,resultId,driverId,constructorId,grid,position,positionText,positionOrder,driverRef,surname,constructorRef
0,1,1,1,1,1,1,1,hamilton,Hamilton,mclaren
1,2,2,2,5,2,2,2,heidfeld,Heidfeld,bmw_sauber
2,3,3,3,7,3,3,3,rosberg,Rosberg,williams
3,4,4,4,11,4,4,4,alonso,Alonso,renault
4,5,5,1,3,5,5,5,kovalainen,Kovalainen,mclaren
